# Load data

In [1]:
from openprompt.data_utils import InputExample
from openprompt import PromptForClassification
from openprompt import PromptDataLoader
from openprompt.prompts import SoftTemplate
from openprompt.plms import load_plm
from openprompt.prompts import ManualVerbalizer
import pandas

# Load data
train_tsvreader = pandas.read_csv("train.tsv", sep = '\t')
# print(train_tsvreader["sentence"][0])
# print(train_tsvreader["label"][0])
train_data = []
# [TODO] 
# Load training data
train_label = []
for i in range(len(train_tsvreader)):
    train_tmp = InputExample(text_a = train_tsvreader["sentence"][i], label = int(train_tsvreader["label"][i]))
    train_data.append(train_tmp)
    train_label.append(int(train_tsvreader["label"][i]))
# print(train_data)

valid_tsvreader = pandas.read_csv("test.tsv", sep = '\t')
test_data = []
# [TODO] 
# Load testing data
test_label = []
for i in range(len(valid_tsvreader)):
    test_tmp = InputExample(text_a = valid_tsvreader["sentence"][i], label = int(valid_tsvreader["label"][i]))
    test_data.append(test_tmp)
    test_label.append(int(valid_tsvreader["label"][i]))
# print(test_data)

# Define model and template

In [2]:
plm, tokenizer, model_config, WrapperClass = load_plm('bert', 'bert-base-uncased')

template    = '{"placeholder":"text_a"} is so {"mask"}.'  # [TODO] Define a template
soft_tokens = 20   # [TODO] number of soft tokens
mytemplate  = SoftTemplate(model = plm, text = template, tokenizer = tokenizer,
                    num_tokens = soft_tokens, initialize_from_vocab = True)

classes = [ # There are two classes in Sentiment Analysis, one for negative and one for positive
    "0",
    "1"
]
promptVerbalizer = ManualVerbalizer(
    classes = classes,
    label_words = {
        "0": ["bad","terrible","disgusting","horrible"],
        "1": ["good", "wonderful", "great","excellent"],
    },
    tokenizer = tokenizer,
)

prompt_model = PromptForClassification(plm = plm, template = mytemplate, verbalizer = promptVerbalizer, freeze_plm = True)    #[TODO] Combine Template and Verbalizer into a PromptModel
prompt_model=  prompt_model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Define DataLoader

In [3]:
train_dataloader = PromptDataLoader(dataset = train_data, template = mytemplate, tokenizer = tokenizer,
    tokenizer_wrapper_class = WrapperClass, max_seq_length = 256, decoder_max_length = 3,
    batch_size = 4, shuffle = True, teacher_forcing = False, predict_eos_token = False,
    truncate_method = "head"
    #[TODO] 
)
valid_dataloader = PromptDataLoader(dataset = test_data, template = mytemplate, tokenizer = tokenizer,
    tokenizer_wrapper_class = WrapperClass, max_seq_length = 256, decoder_max_length = 3,
    batch_size = 4, shuffle = True, teacher_forcing = False, predict_eos_token = False,
    truncate_method = "head"
    #[TODO] 
)

tokenizing: 400it [00:00, 1523.39it/s]
tokenizing: 100it [00:00, 1257.01it/s]


# Train and inference

In [4]:
import torch
from transformers import (
    AdamW
)
no_decay = ["bias", "LayerNorm.weight"]
criterion = torch.nn.CrossEntropyLoss()
optimizer_grouped_parameters = [
    {"params": [p for n, p in prompt_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {"params": [p for n, p in prompt_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]# [TODO] freeze parameter of encoder, only update soft tokens 

# Using different optimizer for prompt parameters and model parameters
optimizer_grouped_parameters1 = [
    {'params': [p for n,p in prompt_model.template.named_parameters() if "raw_embedding" not in n]}
]
optimizer = AdamW(optimizer_grouped_parameters, lr = 3e-3) # [TODO])
optimizer1 = AdamW(optimizer_grouped_parameters1, lr = 3e-4)

for epoch in range(15): # [TODO] number of epochs):
    # [TODO]
    tot_loss = 0.0
    count = 0
    for step, inputs in enumerate(train_dataloader):
        inputs = inputs.cuda()
        predict = prompt_model(inputs)
        loss = criterion(predict, inputs["label"])
        loss.backward()
        tot_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        optimizer1.step()
        optimizer1.zero_grad()
        count += 1
    print("Epoch {}, average loss: {}".format(epoch + 1, tot_loss / count))
train_len = len(train_label)*0.8
test_len = len(test_label)*0.8
def evaluate(dataloader):
    prompt_model.eval()
    total_loss = 0
    pred_list = []

    with torch.no_grad():
        for _, inputs in enumerate(dataloader):
            inputs = inputs.cuda()
            pred = prompt_model(inputs)
            pred_list.extend(torch.argmax(pred, dim = -1).cpu().tolist())
            total_loss += criterion(pred, inputs["label"])
    return total_loss, pred_list

def print_result(train_loss, train_acc, test_loss, test_acc):
    """RESULT"""
    print("FINAL RESULT:")
    print("=" * 25)
    print("Training Data:")
    print("Final Loss: %.5f" %train_loss) 
    print("Final Accuracy: %.5f" %train_acc) 
    print("=" * 25)
    print("Testing Data:")
    print("Final Loss: %.5f" %test_loss)    
    print("Final Accuracy: %.5f" %test_acc)      
    print("=" * 25)      
    """RESULT"""

train_loss, train_pred = evaluate(train_dataloader)
test_loss, test_pred = evaluate(valid_dataloader)

train_acc = sum([int(i == j) for i, j in zip(train_pred, train_label)])/train_len
test_acc = sum([int(i == j) for i, j in zip(test_pred, test_label)])/test_len
print_result(train_loss, train_acc, test_loss, test_acc)

c:\Users\abc89\anaconda3\envs\pytorch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, average loss: 0.4484763199463487
Epoch 2, average loss: 0.38302979096770284
Epoch 3, average loss: 0.2998768799006939
Epoch 4, average loss: 0.27077757283113896
Epoch 5, average loss: 0.23960866548120976
Epoch 6, average loss: 0.1903487076656893
Epoch 7, average loss: 0.1496634561708197
Epoch 8, average loss: 0.13049698527436704
Epoch 9, average loss: 0.09951501901261509
Epoch 10, average loss: 0.08302601332776248
Epoch 11, average loss: 0.08575434601632878
Epoch 12, average loss: 0.056581757572712374
Epoch 13, average loss: 0.0389613206370268
Epoch 14, average loss: 0.014625103387224954
Epoch 15, average loss: 0.00764112455479335
FINAL RESULT:
Training Data:
Final Loss: 0.46334
Final Accuracy: 0.61250
Testing Data:
Final Loss: 14.73317
Final Accuracy: 0.63750


: 